In [1]:
import os

In [2]:
%pwd

'd:\\Full Stack Data Science\\Time Series Analysis\\MAJOR PROJECT\\SMDF\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Full Stack Data Science\\Time Series Analysis\\MAJOR PROJECT\\SMDF'

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv("artifacts/data_ingestion/salesDaily.csv")
data.head()

,Unnamed: 0,datum,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06
0,0,2014-01-02,0.0,3.67,3.4,32.40,7.0,0.0,0.0,2.0
1,1,2014-01-03,8.0,4.00,4.4,50.60,16.0,0.0,20.0,4.0
2,2,2014-01-04,2.0,1.00,6.5,61.85,10.0,0.0,9.0,1.0
3,3,2014-01-05,4.0,3.00,7.0,41.10,8.0,0.0,3.0,0.0
4,4,2014-01-06,5.0,1.00,4.5,21.70,16.0,2.0,6.0,2.0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2106 entries, 0 to 2105
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  2106 non-null   int64  
 1   datum       2106 non-null   object 
 2   M01AB       2106 non-null   float64
 3   M01AE       2106 non-null   float64
 4   N02BA       2106 non-null   float64
 5   N02BE       2106 non-null   float64
 6   N05B        2106 non-null   float64
 7   N05C        2106 non-null   float64
 8   R03         2106 non-null   float64
 9   R06         2106 non-null   float64
dtypes: float64(8), int64(1), object(1)
memory usage: 164.7+ KB


In [8]:
data.isnull().sum()

Unnamed: 0    0
datum         0
M01AB         0
M01AE         0
N02BA         0
N02BE         0
N05B          0
N05C          0
R03           0
R06           0
dtype: int64

In [9]:
data.shape

(2106, 10)

In [10]:
data.columns

Index(['Unnamed: 0', 'datum', 'M01AB', 'M01AE', 'N02BA', 'N02BE', 'N05B',
       'N05C', 'R03', 'R06'],
      dtype='object')

In [11]:
data.drop(columns=['Unnamed: 0'],inplace=True)

In [13]:
dict(data.dtypes)

{'datum': dtype('O'),
 'M01AB': dtype('float64'),
 'M01AE': dtype('float64'),
 'N02BA': dtype('float64'),
 'N02BE': dtype('float64'),
 'N05B': dtype('float64'),
 'N05C': dtype('float64'),
 'R03': dtype('float64'),
 'R06': dtype('float64')}

In [15]:
data.to_csv("artifacts/data_ingestion/salesDaily.csv",index=False)

In [25]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [26]:
from SMDF.constants import *
from SMDF.utils.common import read_yaml, create_directories

In [27]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [28]:
import os
from SMDF.logging import logger

In [29]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    
    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [30]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-06-28 08:06:14,757: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-28 08:06:14,759: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-28 08:06:14,763: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-06-28 08:06:14,765: INFO: common: created directory at: artifacts]
[2023-06-28 08:06:14,767: INFO: common: created directory at: artifacts/data_validation]
